In [0]:
import pyspark.sql.functions as f
import pyspark.sql.utils
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
from datetime import timedelta, datetime

In [0]:
startdate = (datetime.now() - timedelta(1)).strftime("%Y-%m-%d")
enddate = (datetime.now() - timedelta(1)).strftime("%Y-%m-%d")
dates = [startdate,enddate]
dates

['2024-08-20', '2024-08-20']

In [0]:
# Reading from Delta Lake as a bulk
file_path = "s3://zalando-datalake-binary/test_raw_data"
raw_df = (
        spark.read
        .format("delta")
        .load(file_path)
        .withColumn("source", f.lit("raw"))
        .filter(f.col("dt").between(*dates))
        .limit(4)
        )

raw_df.display()

In [0]:
# Write as a bulk to Delta Lake
save_path = "/team/test_target_data"

(
    raw_df
    .write.format("delta")
    .mode("overwrite")
    .partitionBy("dt")
    .option("replaceWhere", f"dt >= '{start_date}' AND dt <= '{end_date}'")
    .save(save_path)
    
)